# PyWeather
## Import and Setup

In [11]:
# Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from citipy import citipy as cp
from IPython.display import display, HTML

import config
import auth

errors = []
if auth.owm_api_key == '':
    errors.append('ERROR! Openweathermap API key is not configured. Open \'auth.py\' file and set up your API key')
if auth.gmaps_api_key == '':
    errors.append('ERROR! Google Maps API key is not configured. Open \'auth.py\' file and set up your API key')

if len(errors) > 0:
    error_message = '\\n'.join(errors)
    js = f'<script>alert("{error_message}");</script>'
    display(HTML(js))
    exit()

## Pick Random Citieis

Our goal here is to randomly pick N cities so that their locations should be evenly distributed across the whole range of latitudes. While technically we could just use a range of -90 to 90, take N random points from this range and get nearest citieis, there are risks in taking too many values from the range where no cities are located and we could end up taking closest cities that will be densed across several points.

Instead I will use the following algorithm

In [2]:
# Out goal is to randomly pick N cities which locations should be evenly distributed across all range of latitudes
# While technically we could just use a range of -90 to 90, take N random points from this range and get nearest cities
# I will use the following algorithm:
# 1. Take all cities with their coordinates from citipy library and sort them by latitude
# 2. Take the min and max available latitudes (so we don't )
# 2. Split them into the N lists
# 3. Randomly take one city from every list
# This way we evenly pick cities across all range of the available latitudes while still having an element of chance

# First, lets get the list of cities sorted by latitued
all_cities = sorted(cp.WORLD_CITIES_DICT.items(), key=(lambda x: x[0]))
# Lets print first 10 values
all_cities[:10]

AttributeError: module 'citipy' has no attribute 'citipy'

In [ ]:
# Now lets split them into buckets N cities each (except for the last one)
